<a href="https://colab.research.google.com/github/srivardhini-reddy/Emotion-detection-using-ROBERTA.ipynb/blob/main/Emotion%20detection%20using%20ROBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from google.colab import drive
from pathlib import Path



In [2]:
if 'google.colab' in str(get_ipython()):
    drive.mount("/content/drive", force_remount= True)

Mounted at /content/drive


In [3]:
!pip install datasets
!pip install emoji -qq
!pip install evaluate -qq
!pip install transformers==4.28.0 -qq
!pip install accelerate==0.22.0 -qq
!pip install transformers[torch] -qq
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

import torch
from torch.nn.functional import sigmoid

from sklearn.model_selection import train_test_split

from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer

from transformers import DistilBertTokenizer, DistilBertModel
from transformers import DistilBertForSequenceClassification

from transformers import Trainer, TrainingArguments

import evaluate

from datasets import Dataset

import re
import emoji

import wandb

# from datasets import Dataset, DatasetDict
from sklearn.metrics import f1_score, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df = pd.read_csv('/content/train.csv')


In [7]:
df['labels'] = df[['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']].values.tolist()
df['labels'] = df['labels'].apply(lambda labels: [float(x) for x in labels])

In [8]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [9]:
accuracy_metric = evaluate.load('accuracy', 'multilabel')
f1 = evaluate.load('f1','multilabel')


def compute_metrics(eval_pred):
    # accuracy_metric = evaluate.load('accuracy', 'multilabel')

    logits, labels = eval_pred
    # print(logits.shape)
    preds = (logits > 0).astype(int)
    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1_micro = f1.compute(predictions=preds, references=labels, average='micro')
    f1_macro = f1.compute(predictions=preds, references=labels, average='macro')
    return {'f1_micro':f1_micro['f1'],
            'f1_macro':f1_macro['f1'],
            'accuracy':accuracy['accuracy'],
            }

In [12]:
from transformers import RobertaTokenizer, RobertaModel , RobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
model = RobertaModel.from_pretrained('distilroberta-base')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
label_columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']



model = RobertaForSequenceClassification.from_pretrained("distilroberta-base",     num_labels=len(label_columns),
    problem_type="multi_label_classification")


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias',

In [14]:
def preprocess_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True, max_length=128)

In [15]:
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

In [16]:
valid_dataset = Dataset.from_pandas(val_df)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

In [25]:

# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs = 5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     weight_decay=0.01,
#     evaluation_strategy = "epoch",
#     logging_dir = "./logs"
# )

training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size = 16,
    output_dir = r'/content/model folder',
    evaluation_strategy = 'steps',
    eval_steps = 200,
    save_strategy = "steps",
    save_steps = 200,
    load_best_model_at_end = True,
    save_total_limit=2,
    metric_for_best_model = "accuracy",
    greater_is_better = True,
    logging_strategy = 'steps',
    logging_steps = 200,
    report_to = 'wandb',
    run_name = 'multilabel_hf_distillroberta_trainer'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

In [19]:
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
%env WANDB_PROJECT = nlp_course_spring_2024-emtoicons-analysis-hf-trainer

env: WANDB_PROJECT=nlp_course_spring_2024-emtoicons-analysis-hf-trainer


In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
200,0.322200,0.341383,0.607113,0.427728,0.230421
400,0.332300,0.363907,0.626674,0.473437,0.205825
600,0.286900,0.336709,0.629590,0.483937,0.227184
800,0.271600,0.335912,0.661499,0.539599,0.225890
1000,0.243700,0.325160,0.660668,0.518299,0.243366
1200,0.237300,0.330129,0.662867,0.536660,0.243366
1400,0.211000,0.328153,0.670004,0.550127,0.255016
1600,0.210500,0.336695,0.673210,0.549387,0.250485
1800,0.193100,0.334609,0.672767,0.549294,0.247896


TrainOutput(global_step=1935, training_loss=0.25191220818255916, metrics={'train_runtime': 142.0007, 'train_samples_per_second': 217.569, 'train_steps_per_second': 13.627, 'total_flos': 1023309287427840.0, 'train_loss': 0.25191220818255916, 'epoch': 5.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.32815316319465637,
 'eval_f1_micro': 0.6700044176115447,
 'eval_f1_macro': 0.5501273308583662,
 'eval_accuracy': 0.2550161812297735,
 'eval_runtime': 2.1328,
 'eval_samples_per_second': 724.394,
 'eval_steps_per_second': 45.48,
 'epoch': 5.0}

In [28]:
valid_output = trainer.predict(valid_dataset)

In [29]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [30]:
valid_logits = valid_output.predictions[0] if isinstance(valid_output.predictions, tuple) else valid_output.predictions

valid_preds = (valid_logits>0).astype(int)
valid_labels = np.array(valid_output.label_ids).astype(int)

In [31]:
valid_preds.shape

(1545, 11)

In [32]:
best_distilroberta_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_distilroberta_model_checkpoint_step}.")

The best model was saved at step 1400.


In [33]:
wandb.finish()

eval/accuracy,▁▅▂▅▅▇▇█▇▇█
eval/f1_macro,▁▄▅▆█▇█████
eval/f1_micro,▁▄▅▆▇▇▇████
eval/loss,█▄█▃▃▁▂▂▃▃▂
eval/runtime,▁▁▁▁█▁▁▁▁▁▃
eval/samples_per_second,▇█▇█▁█████▆
eval/steps_per_second,▇█▇█▁█████▆
train/epoch,▁▁▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇▇▇██
train/global_step,▁▁▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇▇▇██
train/learning_rate,██▇▆▅▄▄▃▂▁
train/loss,█▅▅▄▃▃▂▂▂▁


In [37]:
model_distilroberta_checkpoint = '/content/model folder/checkpoint-1400'
model_distilroberta_checkpoint

'/content/model folder/checkpoint-1400'

In [38]:
model_distil_roberta = RobertaForSequenceClassification.from_pretrained(model_distilroberta_checkpoint, num_labels = 11)

In [40]:
model_distil_roberta

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [42]:
test_df = pd.read_csv('/content/test.csv')

In [43]:
label_columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
for label in label_columns:
    test_df[label] = test_df[label].apply(lambda x: 0 if x == 'NONE' else 1)
test_df['labels'] = test_df[label_columns].values.tolist()
test_df['labels'] = test_df['labels'].apply(lambda labels: [float(label) for label in labels])

In [44]:
# Convert DataFrame to Hugging Face dataset format
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True, max_length=128)

test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [45]:
training_args = TrainingArguments(
    output_dir=r'/content/model folder',
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to='wandb',
    run_name='multilabel_hf_distillroberta_test',
)

In [46]:
trainer_test = Trainer(
    model=model_distil_roberta,
    args=training_args,
    eval_dataset= test_dataset,
    compute_metrics=compute_metrics,)

In [47]:
distilroberta_test_results = trainer_test.evaluate()

In [48]:
distilroberta_test_results

{'eval_loss': 2.7453391551971436,
 'eval_f1_micro': 0.3213616782168945,
 'eval_f1_macro': 0.29719875413750835,
 'eval_accuracy': 0.0,
 'eval_runtime': 4.9759,
 'eval_samples_per_second': 654.961,
 'eval_steps_per_second': 81.996}

In [49]:
distilroberta_test_output = trainer_test.predict(test_dataset)
distilroberta_test_output

PredictionOutput(predictions=array([[ 2.0029533 , -3.0531764 ,  1.6098036 , ..., -2.316017  ,
        -4.292286  , -4.261031  ],
       [ 1.830505  , -3.7078664 ,  2.3937309 , ...,  0.23937517,
        -4.5188246 , -5.5818706 ],
       [ 3.6884387 , -4.3390965 ,  1.6745567 , ..., -0.84157634,
        -4.850451  , -5.668307  ],
       ...,
       [ 4.2895994 , -2.6781008 ,  2.4678771 , ..., -2.8395712 ,
        -4.388921  , -4.6455417 ],
       [-3.9181967 , -1.724125  , -4.2174983 , ..., -4.7834473 ,
        -1.5242543 , -2.0789363 ],
       [-0.48082215, -2.4469266 , -1.2212557 , ..., -2.6178741 ,
        -3.3659725 , -3.424029  ]], dtype=float32), label_ids=array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32), metrics={'test_loss': 2.7453391551971436, 'test_f1_micro': 0.32136167821689

In [55]:
distilroberta_test_preds =distilroberta_test_output.predictions

In [56]:
distilroberta_test_preds

array([[ 2.0029533 , -3.0531764 ,  1.6098036 , ..., -2.316017  ,
        -4.292286  , -4.261031  ],
       [ 1.830505  , -3.7078664 ,  2.3937309 , ...,  0.23937517,
        -4.5188246 , -5.5818706 ],
       [ 3.6884387 , -4.3390965 ,  1.6745567 , ..., -0.84157634,
        -4.850451  , -5.668307  ],
       ...,
       [ 4.2895994 , -2.6781008 ,  2.4678771 , ..., -2.8395712 ,
        -4.388921  , -4.6455417 ],
       [-3.9181967 , -1.724125  , -4.2174983 , ..., -4.7834473 ,
        -1.5242543 , -2.0789363 ],
       [-0.48082215, -2.4469266 , -1.2212557 , ..., -2.6178741 ,
        -3.3659725 , -3.424029  ]], dtype=float32)

In [57]:
distilroberta_test_preds = torch.tensor(distilroberta_test_preds)

In [58]:
def coversion(predictions, threshold=0.25):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    results = np.zeros(probs.shape)
    results[np.where(probs >= threshold)] = 1
    return results

In [59]:
distilroberta_test_output_final = coversion(distilroberta_test_preds)


In [60]:
distilroberta_test_output_final

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [61]:
df_test = pd.read_csv("/content/test.csv")

In [62]:
df_test[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = distilroberta_test_output_final

In [63]:
df_test.drop(["Tweet"],axis =1, inplace = True)

In [66]:
df_test.to_csv('/content/distilroberta-test.csv')

In [68]:
mapped_labels =[]
for prediction_row in distilroberta_test_output_final:
    mapped_row = [label for label, pred in zip(label, prediction_row) if pred]
    mapped_labels.append(mapped_row)

In [69]:
from pprint import pprint
pprint(mapped_labels[:10])

[['t', 'u', 's'],
 ['t', 'u'],
 ['t', 'u'],
 ['r', 't'],
 ['s'],
 ['t', 'r', 'u', 's'],
 ['r', 't'],
 ['t'],
 ['t'],
 ['s']]
